# OpenWeather

In [1]:
import requests
import time

In [27]:
import requests

class BaseAPIFetcher:
    """
    Classe de base pour interroger des APIs.
    Peut être étendue pour intégrer différentes APIs avec des paramètres spécifiques.
    """
    def __init__(self, base_url, api_key=None, api_value=None):
        self.base_url = base_url
        self.api_key = api_key
        self.api_value = api_value

    def fetch(self, params=None, headers=None):
        headers = headers or {}
        params = params or {}

        if self.api_key and self.api_value:
            params[self.api_key] = self.api_value

        response = requests.get(self.base_url, params=params, headers=headers)
        
        try:
            data = response.json()
        except ValueError as e:
            print("Erreur lors du parsing JSON :", e)
            return None

        if response.status_code == 200:
            return data
        else:
            print(f"Erreur API ({response.status_code}) :", data)
            return None


class OpenWeatherMapHistoricalFetcher(BaseAPIFetcher):
    """
    Classe dédiée à l'API OpenWeatherMap (historique).
    """
    def __init__(self, api_key):
        base_url = "https://history.openweathermap.org/data/2.5/history/city"
        # "appid" est la clé attendue par OpenWeatherMap
        super().__init__(base_url, api_key="appid", api_value=api_key)

    def get_historical_data(self, lat, lon, start=None, end=None, cnt=None):
        params = {
            "lat": lat,
            "lon": lon,
            "type": "hour",  # requis par l'API
        }

        if start:
            params["start"] = start
        if end:
            params["end"] = end
        if cnt:
            params["cnt"] = cnt

        return self.fetch(params=params)


In [28]:
class OpenWeatherMapDayForecastFetcher(BaseAPIFetcher):
    """
    Client pour l'API de prévisions météo d'OpenWeatherMap.
    """
    def __init__(self, api_key):
        base_url = "http://api.openweathermap.org/data/2.5/forecast"
        super().__init__(base_url, api_key="appid", api_value=api_key)

    def get_day_forecast_data_by_coordinates(self, lat, lon, units="metric"):
        """
        Récupère les prévisions météo pour des coordonnées géographiques.
        :param lat: Latitude
        :param lon: Longitude
        :param units: Unités de mesure (par défaut "metric")
        :return: Données météo
        """
        params = {
            "lat": lat,
            "lon": lon,
            "units": units
        }
        return self.fetch(params=params)

    def get_day_forecast_data_by_zip(self, zip_code, units="metric"):
        """
        Récupère les prévisions météo pour un code postal.
        :param zip_code: Code postal
        :param units: Unités de mesure (par défaut "metric")
        :return: Données météo
        """
        params = {
            "zip": zip_code,
            "units": units
        }
        return self.fetch(params=params)

    def get_day_forecast_data_by_city_id(self, city_id, units="metric"):
        """
        Récupère les prévisions météo pour une ville donnée par son ID.
        :param city_id: ID de la ville
        :param units: Unités de mesure (par défaut "metric")
        :return: Données météo
        """
        params = {
            "id": city_id,
            "units": units
        }
        return self.fetch(params=params)

    def get_day_forecast_data_by_city_name(self, city_name, units="metric"):
        """
        Récupère les prévisions météo pour une ville donnée par son nom.
        :param city_name: Nom de la ville
        :param units: Unités de mesure (par défaut "metric")
        :return: Données météo
        """
        params = {
            "q": city_name,
            "units": units
        }
        return self.fetch(params=params)


In [29]:
def to_unix_timestamp(date_str, format="%Y-%m-%d"):
    """
    Convertit une date string (ex: '2024-03-01') en timestamp Unix.
    """
    return int(time.mktime(time.strptime(date_str, format)))

In [30]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
api_key = os.getenv("OPEN_WEATHER_API_KEY")
api_key

'24c85c33a7f10723da7c9427b24e84a6'

## Historical API Call 

In [34]:
lat, lon = 34.0901, -118.4065
start = to_unix_timestamp("2025-04-19")
end = to_unix_timestamp("2025-04-20")

print("start", start)
print("end", end)

start 1745013600
end 1745100000


In [35]:
fetcher = OpenWeatherMapHistoricalFetcher(api_key)
data = fetcher.get_historical_data(lat, lon, start=start, end=end)

In [36]:
print(data)

{'message': 'Count: 25', 'cod': '200', 'city_id': 1, 'calctime': 0.007142652, 'cnt': 25, 'list': [{'dt': 1745013600, 'main': {'temp': 289.57, 'feels_like': 288.89, 'pressure': 1014, 'humidity': 62, 'temp_min': 288.49, 'temp_max': 291.19}, 'wind': {'speed': 6.17, 'deg': 250}, 'clouds': {'all': 40}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]}, {'dt': 1745017200, 'main': {'temp': 289.41, 'feels_like': 288.71, 'pressure': 1014, 'humidity': 62, 'temp_min': 288.22, 'temp_max': 291.25}, 'wind': {'speed': 5.14, 'deg': 270}, 'clouds': {'all': 20}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}]}, {'dt': 1745020800, 'main': {'temp': 289.38, 'feels_like': 288.68, 'pressure': 1014, 'humidity': 62, 'temp_min': 288.23, 'temp_max': 290.62}, 'wind': {'speed': 4.63, 'deg': 250}, 'clouds': {'all': 40}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}]}, {'dt': 1745024400, '

## Day API Call

In [37]:
city_name = "London"
fetcher = OpenWeatherMapDayForecastFetcher(api_key)

In [38]:
data = fetcher.get_day_forecast_data_by_coordinates(lat, lon)
print(data)

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1745182800, 'main': {'temp': 18.64, 'feels_like': 18.07, 'temp_min': 18.64, 'temp_max': 21.57, 'pressure': 1017, 'sea_level': 1017, 'grnd_level': 997, 'humidity': 58, 'temp_kf': -2.93}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'clouds': {'all': 0}, 'wind': {'speed': 3.78, 'deg': 210, 'gust': 2.92}, 'visibility': 9106, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-04-20 21:00:00'}, {'dt': 1745193600, 'main': {'temp': 18.59, 'feels_like': 17.91, 'temp_min': 18.49, 'temp_max': 18.59, 'pressure': 1017, 'sea_level': 1017, 'grnd_level': 995, 'humidity': 54, 'temp_kf': 0.1}, 'weather': [{'id': 801, 'main': 'Clouds', 'description': 'few clouds', 'icon': '02d'}], 'clouds': {'all': 23}, 'wind': {'speed': 3.56, 'deg': 206, 'gust': 3.5}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2025-04-21 00:00:00'}, {'dt': 1745204400, 'main': {'temp': 17.37, 'feels_like': 16.52, 'temp_min': 

In [39]:
API_KEY = "24c85c33a7f10723da7c9427b24e84a6"
CITY = "Paris"
#start_date = datetime.now() - timedelta(5)
#URL = f"https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=48.8566&lon=2.3522&dt={int(start_date.timestamp())}&appid={API_KEY}&units=metric"
URL = f"http://api.openweathermap.org/data/2.5/forecast?q={CITY}&appid={API_KEY}&units=metric"


In [40]:

response = requests.get(URL)
weather_data = response.json()

In [41]:
weather_data

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1745182800,
   'main': {'temp': 10.79,
    'feels_like': 10.3,
    'temp_min': 10.79,
    'temp_max': 10.79,
    'pressure': 1006,
    'sea_level': 1006,
    'grnd_level': 996,
    'humidity': 91,
    'temp_kf': 0},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'clouds': {'all': 75},
   'wind': {'speed': 3.61, 'deg': 110, 'gust': 7.43},
   'visibility': 10000,
   'pop': 1,
   'rain': {'3h': 1.24},
   'sys': {'pod': 'n'},
   'dt_txt': '2025-04-20 21:00:00'},
  {'dt': 1745193600,
   'main': {'temp': 10.84,
    'feels_like': 10.33,
    'temp_min': 10.84,
    'temp_max': 10.93,
    'pressure': 1006,
    'sea_level': 1006,
    'grnd_level': 996,
    'humidity': 90,
    'temp_kf': -0.09},
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10n'}],
   'clouds': {'all': 83},
   'wind': {'speed': 3.46, 'deg': 114, 'gust': 7.18},
   '